In [2]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
# call OpenAI API with model name and prompt
def call_openai_api(messages, model_name='gpt-35-turbo', max_token=100, stop=None, n=1, temperature=0.1):
    openai.api_type = "azure"
    openai.api_version = "2023-06-01-preview"
    
    openai.api_key = os.getenv('OPENAI_API_KEY') 
    openai.api_base = os.getenv("OPENAI_API_BASE")

    response = openai.ChatCompletion.create(
        engine=model_name,
        messages=messages,
        temperature=temperature,
        max_tokens=max_token,
        stop=stop,
        n=n,
    )
    return response

model=os.getenv('DEPLOYMENT_NAME')

# Chain of Thought

Experiment results demonstrate Zero-shot-CoT using single prompt template, significantly outperform zero-shot LLM performance on diverse benchmark reasoning tasks.  Without any hand-crafted few-shot examples, e.g. increasing the accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with large InstructGPT model (text-davinci-002).

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

In [49]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are there?
"""
messages=[
    {"role": "user", "content": PROMPT_ZERO_SHOT},
    {"role": "assistant", "content": "The answer (arabic numerals) is "},
]

response = call_openai_api(messages, model_name=model, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

4. 

Half of 16 is 8, so there are 8 golf balls. Half of the golf balls are blue, so there are 4 blue golf balls.


In [45]:
# Still wrong answer with few-shot learning

messages=[
    {"role": "system", "content": "Generate next conversation."},
    {"role": "user", "content": "Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?"},
    {"role": "assistant", "content": "A: The answer is 11."},
    {"role": "user", "content": "Q: A juggler can juggle 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?"},
]

response = call_openai_api(messages, model_name=model, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

A: There are 8 blue golf balls.


In [26]:
# With CoT, the answer is correct

PROMPT_ZERO_SHOT_CoT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: Let’s think step by step.
"""

messages=[
    {"role": "user", "content": PROMPT_ZERO_SHOT_CoT},
]

response = call_openai_api(messages, model_name=model, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

First, we know that the juggler can juggle 16 balls in total. 
Next, we know that half of the balls are golf balls, so there are 8 golf balls. 
Finally, we know that half of the golf balls are blue, so there are 4 blue golf balls. 

Therefore, there are 4 blue golf balls.


In [46]:
# PROMPT_FEW_SHOT_CoT

messages=[
    {"role": "user", "content": """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""},
    {"role": "assistant", "content": """Roger started with 5 balls.\n 2 cans of 3 tennis balls each is 6 tennis balls.\n 5 + 6 = 11. \n The answer is 11."""},
    {"role": "user", "content": """A juggler can juggle 16 balls. Half of the balls are golf balls, and half of the golf balls are blue. How many blue golf balls are there?"""},
]

response = call_openai_api(messages, model_name=model, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

If the juggler has 16 balls, and half of them are golf balls, then there are 16/2 = 8 golf balls.
If half of the golf balls are blue, then there are 8/2 = 4 blue golf balls.
Therefore, there are 4 blue golf balls.


# Program-aided Language Models 


In [54]:
messages=[
    {"role": "user", "content": """Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"""},
    {"role": "assistant", "content": """Roger started with 5 tennis balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."""},
    {"role": "user", "content": """The bakers at the Beverly Hills Bakery baked 200 loaves of bread on Monday morning. They sold 93 loaves
in the morning and 39 loaves in the afternoon. A grocery store returned 6 unsold loaves. How many loaves of bread did they have left?"""},
]

response = call_openai_api(messages, model_name=model, temperature=0, max_token=100)

print(response['choices'][0]["message"]["content"])

The bakers baked 200 loaves of bread, sold 93 in the morning and 39 in the afternoon, so they sold a total of 93 + 39 = 132 loaves.
They had 200 - 132 = 68 loaves of bread left.
Then, the grocery store returned 6 unsold loaves, so the bakers had 68 + 6 = 74 loaves of bread left. Answer: \boxed{74}.


In [56]:
messages=[
    {"role": "user", "content": """Q: Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?"""},
    {"role": "assistant", "content": """A: Roger started with 5 tennis balls.
  tennis_balls = 5
2 cans of 3 tennis balls each is
  bought_balls = 2 * 3 
The answer is
answer = tennis_balls + bought_balls"""},
    {"role": "user", "content": """Q: The bakers at the Beverly Hills Bakery baked 200
loaves of bread on Monday morning. They sold 93 loaves
in the morning and 39 loaves in the afternoon. A grocery
store returned 6 unsold loaves. How many loaves of bread
did they have left?"""},
]

response = call_openai_api(messages, model_name=model, temperature=0, max_token=400)

print(response['choices'][0]["message"]["content"])

# response = call_openai_api(PROMPT_FEW_SHOT_PA, model_name='text-davinci-003', temperature=0, max_token=400)

# print(response['choices'][0]['text'])

The bakers baked 200 loaves of bread.
  baked = 200
They sold 93 loaves in the morning and 39 loaves in the afternoon.
  sold = 93 + 39 = 132
A grocery store returned 6 unsold loaves.
  returned = 6
The number of loaves left is
left = baked - sold + returned


In [ ]:
# 작업중... 하단 생략

# !git clone https://github.com/luyug/pal
# !pip install -e ./pal

In [ ]:
# 작업중... 하단 생략

# import pal
# from pal.prompt import math_prompts

# exec()

# Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)



Provide knowledge, turn knowledge question into reasoning. In general, more knowledge, better result.

3 Contributing factors:

(i) the quality of knowledge, 

(ii) the quantity of knowledge where the performance improves with more knowledge statements, and 

(iii) the strategy for integrating knowledge during inference

In [64]:
PROMPT = """The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""

messages=[
    {"role": "user", "content": PROMPT},
]

response = call_openai_api(messages, model_name=model, max_token=100)

print(response['choices'][0]["message"]["content"])

False. The goal of golf is to have the lowest score possible.


In [65]:
PROMPT = """A tripod is a kind of easel
How many legs does an easel have?
"""
messages=[
    {"role": "user", "content": PROMPT},
]

response = call_openai_api(messages, model_name=model, max_token=100)

print(response['choices'][0]["message"]["content"])

An easel typically has three legs.


## Check out follow 2 examples

In [66]:
# High confidence answer
PROMPT = """Question: Part of golf is trying to get a higher point total than others. Yes or No?

Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.

Explain and Answer: 
"""
messages=[
    {"role": "user", "content": PROMPT},
]

response = call_openai_api(messages, model_name=model, max_token=100)

print(response['choices'][0]["message"]["content"])

No, part of golf is not trying to get a higher point total than others. The objective of golf is to play a set of holes in the least number of strokes. The winner of the game is determined by the total number of strokes taken to complete the round, not by a point system. Each stroke is counted as one point, but the goal is to have the lowest point total possible. Therefore, the statement that part of golf is trying to get a higher point total than others is incorrect


In [71]:
# Low confidence answer. 예문을 더 애매하게 바꿔야... 잘못된 결과가 나올것 같은데... 정상으로 나옴.
PROMPT = """Question: Part of golf is trying to get a higher point total than others. Yes or No?

Knowledge: Golf is a precision club-and-ball sport in which competing players (or golfers) use several types of clubs to hit a ball into a series of holes on a course using the fewest number of strokes. A goal is highest rated with the lowest score calculated by adding up the total number of strokes taken on each hole. The player with the lowest score wins the game.

Explain and Answer:
 
"""
messages=[
    {"role": "user", "content": PROMPT},
]

response = call_openai_api(messages, model_name=model, max_token=100)

print(response['choices'][0]["message"]["content"])

No, part of golf is not trying to get a higher point total than others. In fact, the goal of golf is to have the lowest score possible by hitting the ball into the holes on the course using the fewest number of strokes. The player with the lowest score at the end of the game is the winner. Therefore, it is not about getting a higher point total than others, but rather achieving the lowest score possible.
